## Creating a Regression Model

In this exercise, you will implement a regression model that uses features of a flight to predict how late or early it will arrive.

### Import Spark SQL and Spark ML Libraries

First, import the libraries you will need:

In [5]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

### Load Source Data
The data for this exercise is provided as a CSV file containing details of flights. The data includes specific characteristics (or *features*) for each flight, as well as a *label* column indicating how many minutes late or early the flight arrived.

You will load this data into a DataFrame and display it.

In [6]:
wkdir ='file:///mnt/c/Users/Adura/Google Drive/Projects/Jupyter/SparkMs/data/'
csv = spark.read.csv(wkdir + 'flights.csv', inferSchema=True, header=True)
csv.show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
|        19|        5|     DL|          10397|        15016|      -1|     -19|
|        19|        5|     DL|          15016|        10397|       0|      -1|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|      

### Prepare the Data
Most modeling begins with exhaustive exploration and preparation of the data. In this example, you will simply select a subset of columns to use as *features* as well as the **ArrDelay** column, which will be the *label* your model will predict.

In [7]:
data = csv.select("DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay", "ArrDelay")
data.show()

+----------+---------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+---------------+-------------+--------+--------+
|        19|        5|          11433|        13303|      -3|       1|
|        19|        5|          14869|        12478|       0|      -8|
|        19|        5|          14057|        14869|      -4|     -15|
|        19|        5|          15016|        11433|      28|      24|
|        19|        5|          11193|        12892|      -6|     -11|
|        19|        5|          10397|        15016|      -1|     -19|
|        19|        5|          15016|        10397|       0|      -1|
|        19|        5|          10397|        14869|      15|      24|
|        19|        5|          10397|        10423|      33|      34|
|        19|        5|          11278|        10397|     323|     322|
|        19|        5|          14107|        13487|      -7|     -13|
|     

### Split the Data
It is common practice when building supervised machine learning models to split the source data, using some of it to train the model and reserving some to test the trained model. In this exercise, you will use 70% of the data for training, and reserve 30% for testing.

In [8]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 1891526  Testing Rows: 810692


### Prepare the Training Data
To train the regression model, you need a training data set that includes a vector of numeric features, and a label column. In this exercise, you will use the **VectorAssembler** class to transform the feature columns into a vector, and then rename the **ArrDelay** column to **label**.

In [9]:
assembler = VectorAssembler(inputCols = ["DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay"], outputCol="features")
training = assembler.transform(train).select(col("features"), (col("ArrDelay").cast("Int").alias("label")))
training.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[1.0,1.0,10140.0,...|  -17|
|[1.0,1.0,10140.0,...|   -9|
|[1.0,1.0,10140.0,...|  -11|
|[1.0,1.0,10140.0,...|  -11|
|[1.0,1.0,10140.0,...|  -12|
|[1.0,1.0,10140.0,...|  -10|
|[1.0,1.0,10140.0,...|   23|
|[1.0,1.0,10140.0,...|   41|
|[1.0,1.0,10140.0,...|   -8|
|[1.0,1.0,10140.0,...|   -5|
|[1.0,1.0,10140.0,...|   -1|
|[1.0,1.0,10140.0,...|    6|
|[1.0,1.0,10140.0,...|  -19|
|[1.0,1.0,10140.0,...|  -25|
|[1.0,1.0,10140.0,...|   -1|
|[1.0,1.0,10140.0,...|   -2|
|[1.0,1.0,10140.0,...|   -9|
|[1.0,1.0,10140.0,...|   68|
|[1.0,1.0,10140.0,...|   13|
|[1.0,1.0,10140.0,...|   -5|
+--------------------+-----+
only showing top 20 rows



### Train a Regression Model
Next, you need to train a regression model using the training data. To do this, create an instance of the regression algorithm you want to use and use its **fit** method to train a model based on the training DataFrame. In this exercise, you will use a *Linear Regression* algorithm - though you can use the same technique for any of the regression algorithms supported in the spark.ml API.

In [11]:
lr = LinearRegression(labelCol="label",featuresCol="features", maxIter=10, regParam=0.3)
model = lr.fit(training)
print("Model trained!")

Model trained!


### Prepare the Testing Data
Now that you have a trained model, you can test it using the testing data you reserved previously. First, you need to prepare the testing data in the same way as you did the training data by transforming the feature columns into a vector. This time you'll rename the **ArrDelay** column to **trueLabel**.

In [12]:
testing = assembler.transform(test).select(col("features"), (col("ArrDelay")).cast("Int").alias("trueLabel"))
testing.show()

+--------------------+---------+
|            features|trueLabel|
+--------------------+---------+
|[1.0,1.0,10140.0,...|      -11|
|[1.0,1.0,10140.0,...|      -14|
|[1.0,1.0,10140.0,...|       19|
|[1.0,1.0,10140.0,...|       -5|
|[1.0,1.0,10140.0,...|       -6|
|[1.0,1.0,10140.0,...|       13|
|[1.0,1.0,10140.0,...|      -13|
|[1.0,1.0,10140.0,...|      -11|
|[1.0,1.0,10140.0,...|      -18|
|[1.0,1.0,10140.0,...|       18|
|[1.0,1.0,10140.0,...|       -9|
|[1.0,1.0,10140.0,...|       -8|
|[1.0,1.0,10140.0,...|       14|
|[1.0,1.0,10140.0,...|       -8|
|[1.0,1.0,10140.0,...|        7|
|[1.0,1.0,10140.0,...|       30|
|[1.0,1.0,10140.0,...|       -9|
|[1.0,1.0,10140.0,...|       -6|
|[1.0,1.0,10140.0,...|      -14|
|[1.0,1.0,10140.0,...|      -24|
+--------------------+---------+
only showing top 20 rows



### Test the Model
Now you're ready to use the **transform** method of the model to generate some predictions. You can use this approach to predict arrival delay for flights where the label is unknown; but in this case you are using the test data which includes a known true label value, so you can compare the predicted number of minutes late or early to the actual arrival delay. 

In [13]:
prediction = model.transform(testing)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show()

+--------------------+-------------------+---------+
|            features|         prediction|trueLabel|
+--------------------+-------------------+---------+
|[1.0,1.0,10140.0,...| -7.568107663401861|      -11|
|[1.0,1.0,10140.0,...| -5.768445310216983|      -14|
|[1.0,1.0,10140.0,...| 17.175482276437414|       19|
|[1.0,1.0,10140.0,...| -5.775928185968615|       -5|
|[1.0,1.0,10140.0,...| -4.778366116983641|       -6|
|[1.0,1.0,10140.0,...| 2.2045683659111757|       13|
|[1.0,1.0,10140.0,...|-13.757785260712337|      -13|
|[1.0,1.0,10140.0,...|   -3.7821645708626|      -11|
|[1.0,1.0,10140.0,...|  -6.97734193073291|      -18|
|[1.0,1.0,10140.0,...|  27.93733048374117|       18|
|[1.0,1.0,10140.0,...| -8.133178159555445|       -9|
|[1.0,1.0,10140.0,...| -5.140491952600524|       -8|
|[1.0,1.0,10140.0,...| 2.8400045992792653|       14|
|[1.0,1.0,10140.0,...| -6.138734283017465|       -8|
|[1.0,1.0,10140.0,...|  9.822258820742114|        7|
|[1.0,1.0,10140.0,...|   21.7930036485618|    

Looking at the result, the **prediction** column contains the predicted value for the label, and the **trueLabel** column contains the actual known value from the testing data. It looks like there is some variance between the predictions and the actual values (the individual differences are referred to as *residuals*)- later in this course you'll learn how to measure the accuracy of a model.